In [ ]:
from huggingface_hub import login
login("----")

e:\Iran\Code\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def warn(*args,**kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings("ignore")

In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_openai import ChatOpenAI
import torch
import os

## 1- Initialize the LLM

In [5]:

model_name = "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
gemma_pipeline = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=1024,
    do_sample= True,
    temperature=0.1,  # Lower temperature reduces hallucinations
)
llm = HuggingFacePipeline(pipeline=gemma_pipeline)

# OPENAI_API_KEY="---"
# llm= ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, max_tokens=1024,api_key=OPENAI_API_KEY)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 10.14it/s]
Device set to use cpu


## 2- Define the document Loader

In [9]:
import os
files = ['khadamat_cleaned.csv','pages_cleaned.csv','NEWS_cleaned.csv']
docs= []
for file in files:
    loader = CSVLoader(file_path=os.path.join(data_dir, file),
                       csv_args={
                           'delimiter': ',',
                           'quotechar': '"'},
                       encoding='utf-8-sig',
                       )
    docs.extend(loader.load())



## 3- Define the text splitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=50,
    separators=["\n\n", "\n", "۔", "؟", "!", " "]
)
texts = text_splitter.split_documents(docs)


## 4- Define the embedding model

In [11]:
embedder = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)


## 5- Define the vectore store 

In [12]:
vectorstore = FAISS.from_documents(texts, embedder)

## 6- Define the retriever

In [13]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 2}  # Retrieve top 3 relevant chunks
)
retriever.invoke("چه تسهیلاتی توسط بانک ارائه میشود؟")

[Document(id='10615898-4f24-4bb8-b265-46a65e5cb429', metadata={'source': 'e:\\Iran\\Data\\csv\\khadamat_cleaned.csv', 'row': 17}, page_content='برای مشتریان حقوقی بانک است که خدماتی مانند صدور کارت اعتباری و همچنین اعطای تسهیلات در قالب عقد جعاله و فروش اقساطی برای کارکنان مشتریان حقوقی بانک را با شرایط ویژه دریافت کنند. بیشتر بدانیم » تسهیلات قرض\u200cالحسنه ازدواج تسهیلات قرض\u200cالحسنه ازدواج با هدف حمایت از زوج\u200cهای جوان در قالب قرض\u200cالحسنه پرداخت می\u200cشود. بیشتر بدانیم » تسهیلات قرض\u200cالحسنه فرزندآوری تسهیلات قرض\u200cالحسنه فرزندآوری با توجه به قانون حمایت از خانواده و جوانی جمعیت برای تولد هر فرزند در قالب قرض\u200cالحسنه پرداخت می\u200cشود. بیشتر بدانیم » آدرس کوتاه شده: https://enbank.ir/s/mfalqO'),
 Document(id='ac4b5c5c-a425-47c2-b501-d6c596e2ef39', metadata={'source': 'e:\\Iran\\Data\\csv\\khadamat_cleaned.csv', 'row': 17}, page_content='desc: چچ تسهیلات مشارکت مدنی تسهیلات مشارکت مدنی با هدف کمک به پیشرفت و رشد و رونق پروژه\u200cهای فعالان اقتصادی در نظر گرف

In [14]:
# 6. Create anti-hallucination prompt (Persian/English)
template = """
به عنوان یک دستیار هوشمند، فقط از اطلاعات ارائه شده در بخش زمینه استفاده کن.
اگر پاسخ را نمی‌دانی بگو: 'پاسخ این سوال در دانش من موجود نیست'.

زمینه:
{context}

سوال: {question}
پاسخ به فارسی:
""".strip()

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="به عنوان یک دستیار هوشمند، فقط از اطلاعات ارائه شده در بخش زمینه استفاده کن.\nاگر پاسخ را نمی\u200cدانی بگو: 'پاسخ این سوال در دانش من موجود نیست'.\n\nزمینه:\n{context}\n\nسوال: {question}\nپاسخ به فارسی:")

In [15]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)


In [16]:
def get_response(query):
    result = qa_chain.invoke({"query": query})
    
    # Verify answer relevance
    
    return result['result']



In [17]:
user_query = "چه تسهیلاتی توسط بانک ارائه میشود؟"
response = get_response(user_query)
response

"به عنوان یک دستیار هوشمند، فقط از اطلاعات ارائه شده در بخش زمینه استفاده کن.\nاگر پاسخ را نمی\u200cدانی بگو: 'پاسخ این سوال در دانش من موجود نیست'.\n\nزمینه:\nبرای مشتریان حقوقی بانک است که خدماتی مانند صدور کارت اعتباری و همچنین اعطای تسهیلات در قالب عقد جعاله و فروش اقساطی برای کارکنان مشتریان حقوقی بانک را با شرایط ویژه دریافت کنند. بیشتر بدانیم » تسهیلات قرض\u200cالحسنه ازدواج تسهیلات قرض\u200cالحسنه ازدواج با هدف حمایت از زوج\u200cهای جوان در قالب قرض\u200cالحسنه پرداخت می\u200cشود. بیشتر بدانیم » تسهیلات قرض\u200cالحسنه فرزندآوری تسهیلات قرض\u200cالحسنه فرزندآوری با توجه به قانون حمایت از خانواده و جوانی جمعیت برای تولد هر فرزند در قالب قرض\u200cالحسنه پرداخت می\u200cشود. بیشتر بدانیم » آدرس کوتاه شده: https://enbank.ir/s/mfalqO\n\ndesc: چچ تسهیلات مشارکت مدنی تسهیلات مشارکت مدنی با هدف کمک به پیشرفت و رشد و رونق پروژه\u200cهای فعالان اقتصادی در نظر گرفته شده است. بیشتر بدانیم » مضاربه بازرگانان و فعالان بخش تجاری کشور می\u200cتوانند به\u200cمنظور گسترش امور بازرگانی داخلی منابع